In [15]:
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

import nltk

In [35]:
class Embeddings():
    def __init__(self, vocab, emd_dims):
        self.vocab = vocab
        self.embeds = nn.Embedding(len(vocab), emd_dims)
    
    def get_embedding(self, word):
        lookup_tensor = torch.tensor(self.vocab[word], dtype = torch.long)
        return self.embeds(lookup_tensor)

In [7]:
test_df = pd.read_csv('./data/coco/coco_test_all.csv')

In [27]:
data_folder = ['coco','bing','flickr']
dataframes = []
for f in data_folder:
    files = os.listdir('./data/'+f)   
    for path in files:
        if('.csv' in path):
            csv_path = './data/'+f+'/'+path
            print(csv_path)
            df = pd.read_csv(csv_path)
            dataframes.append(df)
len(dataframes)

./data/coco/coco_test_all.csv
./data/coco/coco_train_all.csv
./data/coco/coco_val_all.csv
./data/bing/bing_train_all.csv
./data/bing/bing_test_all.csv
./data/bing/bing_val_all.csv
./data/flickr/flickr_val_all.csv
./data/flickr/flickr_train_all.csv
./data/flickr/flickr_test_all.csv


9

In [29]:
df = pd.concat(dataframes, axis = 0)
len(df)

14815

In [33]:
questions = list(df['questions'])
freq = {}
for q in questions:
    for question in q.split('---'):
        wordlist = nltk.word_tokenize(question)
        for word in wordlist:
            if(word not in freq):
                freq[word] = 1
            else:
                freq[word] += 1
len(freq)

12098

In [34]:
vocab = {}
counter = 0
for key in freq.keys():
    if freq[key]>=3:
        vocab[key] = counter
        counter += 1
len(vocab)

5061

tensor([ 2.7821e+00,  1.1841e+00, -9.3363e-01,  3.2496e-02,  6.7456e-02,
        -1.7605e+00,  8.2250e-01, -1.0765e-02,  3.0308e-01,  1.7597e+00,
        -2.1359e+00,  1.2264e+00,  9.0684e-01, -9.5138e-01,  1.0943e+00,
        -2.8756e-02,  1.0308e+00, -2.3037e-01, -1.9689e+00,  5.7149e-01,
         1.4413e-01,  5.1339e-01,  1.3354e+00,  1.1311e+00, -4.4950e-01,
         7.8457e-02, -1.8697e+00,  1.7421e+00,  1.2825e+00,  1.6702e+00,
        -9.5160e-01, -1.4192e+00,  1.7293e+00, -5.2661e-01,  1.1287e-01,
         6.6286e-01,  1.1682e+00, -5.5004e-01, -2.3742e-01, -1.5798e+00,
        -3.3154e-01,  1.6581e-01, -4.8205e-01,  4.7807e-01,  1.2499e-01,
         1.3876e+00,  2.0622e-01, -5.3768e-02, -4.4470e-01,  7.3125e-01,
         9.1847e-02,  7.1872e-01,  5.7741e-01,  2.0322e-01, -6.1851e-01,
         3.5980e-01,  1.1739e+00,  3.6032e-01,  1.4306e+00,  2.7806e-01,
        -2.9177e-01, -1.6236e+00, -8.5929e-01,  3.1841e-01, -2.7547e-01,
        -1.9578e-01, -1.6194e+00,  1.1771e+00,  1.2